# 6A - Auto-Encoder in 2D

The goal of an auto-encoder is to learn a representation of the input data in a lower-dimensional space, and as been introduced as a non linear generalization of PCA (Principal Component Analysis).
It can be useful for tasks like dimensionality reduction or feature extraction, but also for generative modeling as we will investigate here.

To begin with, we implement a simple auto-encoder based on MLP using PyTorch and investigate the learned latent representation for the purpose of generative modeling. Last we conclude on the need to train a more sophisticated generative model.

---

References: 
- "Neural networks and principal component analysis: Learning from examples without local minima", P Baldi, K Hornik - Neural networks, 1989

- "Nonlinear principal component analysis using autoassociative neural networks.", Kramer, Mark A. AIChE journal, 1991

julien rabin @ greyc.ensicaen.fr 2025

```If the following cell fails to run, please make sure you have installed the required packages and setup a python environment as indicated in the README.md file.```

In [ ]:
import torch 
from torch import nn, Tensor

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
# Function to sample synthetic 2D data
def sample_data(n: int, model = 'moons') -> Tensor:
    data_dim = 2  # Dimension of the data
    if model == 'moons':
        from sklearn.datasets import make_moons
        return Tensor(make_moons(n_samples=n, noise=0.05)[0])
    elif model == 'circles':
        from sklearn.datasets import make_circles
        return Tensor(make_circles(n_samples=n, noise=0.05, factor=0.5)[0])
    elif model == '2gmm':
        n_samples = n
        n = n//3
        X1 = torch.randn(n, data_dim) @ torch.tensor([[.05, -0.02],[-0.02, .4]]) + torch.tensor([[.0, 1.0]]).view(1, data_dim)
        X2 = torch.randn(n_samples - n, data_dim) @ torch.tensor([[.3, 0.05],[0.05, .05]]) + torch.tensor([[-1.0, 0.]]).view(1, data_dim)
        return torch.cat((X1,X2), dim=0)  # Concatenate the two sets of samples
    elif model == 'radial_gmm':
        K = 8
        n_samples = n
        samples_per_component = n_samples // K
        remainder = n_samples % K
        all_samples = []

        for k in range(K):
            radius = 3.
            # Angle for the mean on a circle
            theta = 2 * np.pi * k / K
            cs = np.cos(theta)
            sn = np.sin(theta)

            # Radial direction unit vector
            radial = torch.tensor([cs, sn], dtype=float)#.view(data_dim, 1)
            tangential = torch.tensor([-sn, cs], dtype=float)#.view(data_dim, 1)

            mean = radius * radial
            
            # Covariance matrix: elongated along radial direction
            cov = 0.3 * torch.outer(radial, radial).to(float) + 0.05 * torch.outer(tangential, tangential).to(float)

            # Generate samples
            n = samples_per_component + (remainder if k == K else 0)
            samples = torch.randn(n, data_dim, dtype=float) @ torch.linalg.cholesky(cov).to(float) + mean.to(float)
            all_samples.append(samples)

        return torch.cat(all_samples, dim=0).to(torch.float32)
    else:
        raise ValueError(f"Unknown model: {model}.")


# Part I: AE Setup

### Exercise 1: Implement a simple Auto-Encoder in 2D using the following template.
To illustrate the dimensionality reduction capabilities of auto-encoders, we will use 2D synthetic datasets (`dim=2`) and enforce the latent space to be 1D (`latent_dim=1`). 

In [ ]:
class AE(nn.Module):
    def __init__(self, dim: int = 2, h: int = 64, latent_dim: int = 1): # dim : input data dimension, h : hidden layer size, latent_dim : latent space dimension
        super().__init__()
        
        self.lat_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Linear(dim, latent_dim)) # Output is the latent representation
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, dim))
    
    def forward(self, x: Tensor) -> Tensor: # AE mode : encode then decode
        # Note : x is of shape (batch_size, dim)
        
        # latent representation of shape (batch_size, latent_dim)
        z = self.encode(x)
        
        # Decode the latent space representation to data space
        y = self.decode(z) # Output is of shape (batch_size, dim)
        
        return y
    
    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        # Encode input x to latent space
        z = self.encoder(x)
        return z
    
    def decode(self, z: Tensor) -> Tensor:
        # Decode latent space representation z to data space
        y = self.decoder(z)
        return y
    
    def sample_gauss(self, n: int, mu : Tensor = None, std : Tensor = None) -> Tensor:
        # Sample from the latent space distribution N(mu, std I_d)
        # Note : there is no reason why the learned latent distribution should be gaussian as AE are not generative model and this is not enforced during training !
        if mu is None :
            mu = torch.zeros((n, self.lat_dim))
        if std is None :
            std = torch.ones((n, self.lat_dim))
        assert mu.shape == (n, self.lat_dim) and std.shape == (n, self.lat_dim), "mu and std must have shape (n, latent_dim)"
        # Sample from the latent space distribution N(mu, std I_d)
        z = ...
        return self.decoder(z)
    
    def sample_uniform(self, n: int, low: float = -1.0, high: float = 1.0) -> Tensor:
        assert low < high, "low must be less than high"
        # Sample *uniformly* from the latent space between low and high
        z = ...
        return self.decoder(z)
    

Number of parameters

In [ ]:
model = AE(dim=2, h=64, latent_dim=1)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {n_params}")

Test the (untrained) AE : recontruct data

In [ ]:
def plot_data_comparison(x_data : np.ndarray, x_model : np.ndarray, colors = None):
    if colors is None:
        colors = 'C0' #np.random.rand(x_data.shape[0])
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True)

    # Original data
    axes[0].scatter(x_data[:, 0], x_data[:, 1], s=10, c=colors)
    axes[0].set_title('Data Samples')
    axes[0].set_xlim(-3.0, 3.0)
    axes[0].set_ylim(-3.0, 3.0)

    # VAE reconstruction
    axes[1].scatter(x_model[:, 0], x_model[:, 1], s=10, c=colors)
    axes[1].set_title('AE Output Samples')
    axes[1].set_xlim(-3.0, 3.0)
    axes[1].set_ylim(-3.0, 3.0)

    # Global title
    fig.suptitle("Original vs Model samples", fontsize=24)
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)

    return fig, axes


In [ ]:
def define_colors(x_data: np.ndarray ) -> np.ndarray:
    cmap = plt.get_cmap("rainbow")
    values = x_data[:, 0]  # Use the first dimension for coloring
    normalized = (values - values.min()) / (values.max() - values.min())
    colors = cmap(normalized)
    return colors[:, :3]

You can choose between 'moons', 'circles', '2gmm' and 'radial_gmm' by changing the variable `dataset_name` in the code cell below.

In [ ]:
dataset_name = 'moons' # moons, circles, 2gmm, radial_gmm
x = sample_data(n=1_000, model = dataset_name)  # Generate synthetic data

# reconstruction of the training data
model = AE(dim=2, h=64, latent_dim=1)
y = model(x).detach()

# define colors for the data points
colors = define_colors(x.detach())

fig, axes = plot_data_comparison(x,y, colors=colors)

fig.suptitle("*Untrained* AE reconstruction of data samples", fontsize=24)

### Exercise 2: explain the results obtained with the untrained AE for different MLPs (e.g. w/ or w/o activation layers and hidden layers, with latent dimensions = 1 or >1). What do you observe ? Why ?

### Exercice 3: Test the (untrained) AE : generate samples

As we do no know how to sample the latent space distribution $\nu = E \# \mu$ which is the push-forward by the Encoder $E$ of the data-distribution $\mu$ (with density $p(x)$), we sample random points using a standard distribution $\mathcal N(\mu,\sigma^2 I)$ in the latent space and decode them to generate random samples.

Complete the following code to generate samples from the untrained AE for different values of $\mu$ in the latent space. You can choose the standard deviation $\sigma$ as you want. Analyze the results.

In [ ]:
torch.manual_seed(42)
n = 1_000

for latent_dim in [1,2] :
    
    model = AE(dim=2, h=64, latent_dim=latent_dim)

    fig, ax = plt.subplots(figsize=(10, 10))
    # Generate samples from the untrained AE using a random latent variable
    for t in np.linspace(-10, 10, 10):
        mu = t + torch.zeros((n, latent_dim)) # Mean shifted along the latent dimension
        std = 1 * torch.ones((n, latent_dim)) # fixed Standard deviation
        y = ... # generate samples y = D(z) where z ~ N(mu, std I_d) and D is the AE decoder
        
        ax.scatter(y[:, 0], y[:, 1], s=10, label=f"mu={t:.2f}")
    ax.set_title(f"Sampling Untrained AE with latent dim={latent_dim} for different mu values")
    fig.tight_layout()
    ax.legend()
    plt.show()



# Part II: AE Training

#### Recap on the course: Reconstruction loss for Auto-Encoders

The reconstruction loss function $\mathcal L_{\text{rec}}$ measures how well the model can reconstruct the input data from the latent representation. 
We denote $y=f_\theta(x)$ the reconstruction an input data $x$ using a parametric function  $f_\theta$ (here a MLP neural network with parameters $\theta$).

- Likelihood maximization, or equivalently negative log-likelihood (NLL) minimization, use the following loss function:
$$\mathcal \ell_{\text{rec}} (x,\theta) = - \log p_\theta(y|x) = - \log p_\theta(f_\theta(x)|x) $$
where $p_\theta(y|x)$ is the conditional probability density of observing $y$ given $x$ using $f_\theta$.

- One samples $x$ from a data probability density $p_{\text{data}}$, in order to minimize the **expected** loss:
$$\mathcal L_{\text{rec}} (\theta) = \mathbb E_x \ell_{\text{rec}} (x,\theta) = - \mathbb E_{x \sim p_{\text{data}}} \left[ \log p_\theta(y|x) \right] $$

- In practice (batch optimization and stochastic gradient descent), using a training dataset $\mathcal D = \{(x_i) \sim p_{\text{data}}\}_{i=1}^N$ of $N$ pairs of input and output data, we can write the empirical loss as:
$$\mathcal L_{\text{rec}} (\theta) \approx - \frac{1}{N} \sum_{i=1}^N \log p_\theta(y_i|x_i) $$

- Assuming that the reconstruction error $y-x$ follows a standard gaussian distribution with zero mean and variance $\sigma^2$, we have
$$
	y = f_\theta(x) = x + \sigma \epsilon \sim \mathcal N(x, \sigma^2 I)
$$
where $\epsilon \sim \mathcal N(0, I)$,
we can express the conditional probability as:
$$p_\theta(y|x) = \mathcal N(y; x, \sigma^2 I) = \frac{1}{(2\pi\sigma^2)^{d/2}} \exp\left(-\frac{1}{2\sigma^2} \| y - x \|^2\right)$$
where $d$ is the dimension of the data.

Note that this is equivalent to model the data reconstruction as a gaussian $y \sim \mathcal N(f_\theta(x), \Sigma)$ centered around the prediction $y=f_\theta(x)$ with diagonal covariance matrix $\Sigma = \sigma^2 \mathbf{I}_d$. This means that, as it is often the case with deep generative modeling, the AE is a neural network that predicts the parameters of a distribution (here the mean of a gaussian, the variance being fixed to a constant $\sigma^2$). 

- Using the properties of the gaussian distribution, we can express the negative log likelihood as:
$$\mathcal \ell_{\text{rec}} (x,\theta) = \tfrac{1}{2} \left(\tfrac{1}{\sigma^2}  \| x - y \|^2 + d\log(2\pi) + d\log(\sigma^2) \right)$$

where $x$ and $\sigma^2$ are the parameters of the gaussian distribution that models the reconstruction of $x$ with some variance $\sigma^2$.

In practice, ignoring the constant terms, we can write the reconstruction loss as:
$$\mathcal \ell_{\text{rec}} (x,\theta) = \| x - y \|^2 $$
so that the reconstruction loss is simply the mean squared error (MSE) between the input data $x_i$ batch and the reconstructed data $y_i$.

Note: other choices of distributions can be made depending on the data type (e.g. Bernoulli distribution for binary data, categorical distribution for discrete data, L1 loss for Laplace distribution, etc.). 

### Exercise 4 : Train the AE on the 2D datasets 
Using the following template, visualize :
- the loss function on training data
- the reconstructed data
- the latent representation of the data

In [ ]:
torch.manual_seed(0) # For reproducibility

latent_dim = 1
model = ... # Define the AE model with given latent dimension

learning_rate = 1e-4
optimizer = ... # use torch.optim.Adam for instance
criterion = ... # Reconstruction loss, see above

In [ ]:
n_iter = 1_000
batch_size = 256

rec_losses = []

pbar = tqdm(range(n_iter), desc="Training AE")
for _ in pbar :
    x = ... # Sample a batch of data of size batch_size from the dataset
    y = ... # Forward pass through the AE : reconstruction of x
    
    optimizer.zero_grad()
    rec_loss = criterion(x, y) # Reconstruction loss 
    rec_loss.backward()
    optimizer.step()
    
    rec_losses.append(rec_loss.item())
    
    pbar.set_postfix({
        'Loss'  : f"{rec_losses[-1]:.4f}",
    })

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.semilogy(rec_losses, label='Reconstruction Loss', color='blue')
ax.set_title('Losses during AE Training')
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss Value')
ax.legend()
plt.show()

#### Recontruction of test data
Is the reconstruction good enough ? 

What is the best we can have when the latent dimension is 1 ?

Compare the results with different datasets and architectures (number of hidden layers, number of neurons per layer, activation functions, latent dimension).

In [ ]:
# Generate samples from the training data
x = ...

# reconstruction of the training data : using only the mean of the learned distribution
y = ...
colors = define_colors(x.detach())
fig, axes = plot_data_comparison(x,y, colors=colors)
fig.suptitle("AE reconstruction of data samples", fontsize=24)



#### Latent Distribution of data and comparison with standard gaussian samples
What can we expect from the latent distribution of the data when training an AE ?

What do you observe ?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True)

n = 1_000
# Generate samples from the training data
x = ...

# reconstruction of the training data
z = ... # Encode x to latent space using the trained AE
eps = torch.randn_like(z).detach().numpy() # 1D Standard gaussian samples for comparison

if model.lat_dim >= 2:
    axes[0].scatter(eps[:, 0], eps[:, 1], s=10, color='red')
    axes[0].set_title(f'standard gaussian latent space (first 2 dimensions)')
    axes[0].set_xlim(-3.0, 3.0)
    axes[0].set_ylim(-3.0, 3.0)

    z = z.detach().numpy()
    axes[1].scatter(z[:, 0], z[:, 1], s=10)
    axes[1].set_title(f'AE latent (first 2 dimensions)')
    axes[1].set_xlim(-3.0, 3.0)
    axes[1].set_ylim(-3.0, 3.0)
    
else: # 1D plot
    t = np.linspace(-3,3, 100)
    axes[0].hist(eps[:, 0],bins=t,color='r')
    axes[0].set_title(f'standard gaussian latent space (1D here)')

    z = z.detach().numpy()
    axes[1].hist(z[:, 0],bins=t)
    axes[1].set_title(f'AE latent (1D here)')
    

### Exercice 5 : a (naive) generative model 
What generative distribution is obtained by sampling the latent space with a standard normal distribution ?

In [ ]:
torch.manual_seed(42)
n = 1_000

latent_dim = model.lat_dim

fig, ax = plt.subplots(figsize=(5, 5))
# Generate samples from the untrained AE using a random latent variable
for t in np.linspace(-10, 10, 10):
    mu = t + torch.zeros((n, latent_dim))
    std = 1 * torch.ones((n, latent_dim))
    y = ... # generate samples y = D(z) where z ~ N(mu, std I_d) and D is the AE decoder
    
    ax.scatter(y[:, 0], y[:, 1], s=10, label=f"t={t:.2f}")
ax.set_title(f"Sampling Untrained AE with latent dim={latent_dim} for different mu values")
fig.tight_layout()
ax.legend()
plt.show()

